## Import libraries

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Read csv with questions

In [15]:
question_df = pd.read_csv("../data/survey_results_schema.csv")
question_df


,Column,Question
0,Respondent,Respondent ID number
1,Professional,Which of the following best describes you?
2,ProgramHobby,Do you program as a hobby or contribute to ope...
3,Country,In which country do you currently live?
4,University,"Are you currently enrolled in a formal, degree..."
...,...,...
149,QuestionsInteresting,The questions were interesting
150,QuestionsConfusing,The questions were confusing
151,InterestedAnswers,I'm interested in learning how other developer...
152,Salary,"What is your current annual base salary, befor..."


## Read csv with answers

In [16]:
df = pd.read_csv("../data/survey_results_public.csv")
df.head(10)


,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Student,"Yes, both",New Zealand,"Yes, full-time","Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Disagree,NaN,A bachelor's degree,White or of European descent,Disagree,Agree,Disagree,Agree,NaN,NaN
6,7,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Master's degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...",20 to 99 employees,...,Disagree,Male,A doctoral degree,White or of European descent,Disagree,Agree,Disagree,Agree,NaN,NaN
7,8,Professional developer,"Yes, both",Poland,No,Employed full-time,Master's degree,Computer science or software engineering,All or almost all the time (I'm full-time remote),Fewer than 10 employees,...,Somewhat agree,Male,A master's degree,White or of European descent,Agree,Somewhat agree,Disagree,Agree,NaN,NaN
8,9,Professional developer,"Yes, I program as a hobby",Colombia,"Yes, part-time",Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","5,000 to 9,999 employees",...,Strongly disagree,Male,A bachelor's degree,Hispanic or Latino/Latina,Somewhat agree,Strongly agree,Disagree,Strongly agree,NaN,NaN
9,10,Professional developer,"Yes, I program as a hobby",France,"Yes, full-time","Independent contractor, freelancer, or self-em...",Master's degree,Computer science or software engineering,It's complicated,NaN,...,Disagree,Male,A doctoral degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,NaN,NaN


### Checking unique values

In [17]:
print(df['CompanySize'].unique())
print("\n")
print(df['CompanyType'].unique())
print("\n")
print(df['Currency'].unique())


[nan '20 to 99 employees' '10,000 or more employees' '10 to 19 employees'
 'Fewer than 10 employees' '5,000 to 9,999 employees'
 '100 to 499 employees' '1,000 to 4,999 employees' '500 to 999 employees'
 "I don't know" 'I prefer not to answer']


[nan 'Privately-held limited company, not in startup mode'
 'Publicly-traded corporation'
 'Non-profit/non-governmental organization or private school/university'
 'Government agency or public school/university' 'Pre-series A startup'
 'Venture-funded startup' "I don't know"
 'Sole proprietorship or partnership, not in startup mode'
 'I prefer not to answer' 'State-owned company' 'Something else']


[nan 'British pounds sterling (£)' 'Canadian dollars (C$)'
 'U.S. dollars ($)' 'Euros (€)' 'Brazilian reais (R$)' 'Indian rupees (?)'
 'Polish zloty (zl)' 'Swedish kroner (SEK)' 'Russian rubles (?)'
 'Swiss francs' 'Australian dollars (A$)' 'Mexican pesos (MXN$)'
 'Japanese yen (¥)' 'Chinese yuan renminbi (¥)' 'Singapore dollars (S$)'
 'Bitcoin (btc

### Checking number of rows

In [18]:
df_prof = df[~df["Professional"].isin(["Student", "None of these"])]
print(f"Length of DataFrame: {len(df_prof)} rows")

df_stud = df[df["Professional"] == "Student"]
print(f"Length of DataFrame: {len(df_stud)} rows")

df_none = df[df["Professional"] == "None of these"]
print(f"Length of DataFrame: {len(df_none)} rows")

print(f"Sum of rows: {len(df_prof) + len(df_stud) + len(df_none)}")


Length of DataFrame: 42254 rows
Length of DataFrame: 8224 rows
Length of DataFrame: 914 rows
Sum of rows: 51392


## Preparing dataset

In [19]:
countries = ["Austria", "Belarus", "Belgium", "Bulgaria", "Croatia", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Ireland", "Italy", "Lithuania", "Luxembourg", "Netherlands", "Norway", "Poland", "Portugal","Romania", "Serbia", "Slovak Republic", "Slovenia", "Spain", "Sweden", "Switzerland", "United Kingdom", "United States"]

columns = ["Respondent", "Gender", "Country", "EmploymentStatus", "FormalEducation", "CompanyType",  "CompanySize", "Currency", "Professional", "HaveWorkedLanguage", "WantWorkLanguage",  "JobSatisfaction", "Salary"]

# Filter by country and select rows which we are interested in
df_filtered = df[df["Country"].isin(countries)][columns]

# Split into two groups
df_students = df_filtered[df_filtered["Professional"] == "Student"].reset_index(drop=True)

df_professionals = df_filtered[
    ~df_filtered["Professional"].isin(["Student", "None of these"])
].reset_index(drop=True)

print(f"Filtered: {len(df_filtered)} rows")
print(f"Students: {len(df_students)} rows")
print(f"Professionals: {len(df_professionals)} rows")
print(f"Sum of rows: {len(df_students) + len(df_professionals)}")


Filtered: 31863 rows
Students: 4536 rows
Professionals: 26703 rows
Sum of rows: 31239


In [20]:
df_professionals["Currency"].unique()


array(['British pounds sterling (£)', nan, 'U.S. dollars ($)',
       'Euros (€)', 'Polish zloty (zl)', 'Swedish kroner (SEK)',
       'Swiss francs', 'Australian dollars (A$)',
       'Chinese yuan renminbi (¥)', 'Japanese yen (¥)',
       'Indian rupees (?)', 'Canadian dollars (C$)', 'Bitcoin (btc)',
       'Mexican pesos (MXN$)', 'Russian rubles (?)',
       'Brazilian reais (R$)'], dtype=object)

### Further changes
We will focus only on group of professionals due to the fact that only those group of respondents can give us valuable information about salaries and job satisfaction levels. 

### Additional functions to organize data

In [21]:
# Split multiple-choice fields into binary columns
def process_multichoice(df, column, prefix):
    df[column] = df[column].fillna('')
    languages = df[column].str.get_dummies(sep=';')
    languages.columns = [f"{prefix}_{col.strip()}" for col in languages.columns]
    return languages

# Process languages
have_lang = process_multichoice(df_professionals, "HaveWorkedLanguage", "HWL")
want_lang = process_multichoice(df_professionals, "WantWorkLanguage", "WWL")

# Combine with original DataFrame
df_professionals = pd.concat([df, have_lang, want_lang], axis=1)
df_professionals.drop(["HaveWorkedLanguage", "WantWorkLanguage"], axis=1, inplace=True)


### Dictionaries

In [22]:
company_size_num = {"Fewer than 10 employees" : 1, "10 to 19 employees" : 2, "20 to 99 employees": 3, "100 to 499 employees" : 4, 
                    "500 to 999 employees":5, "1,000 to 4,999 employees": 6, "5,000 to 9,999 employees":7, "10,000 or more employees":8 }

currencies_to_dollar = {"British pounds sterling (£)" : 1.33 ,"Euros (€)" : 1.13, "Polish zloty (zl)" : 0.27, "Swedish kroner (SEK)" : 0.1, "Swiss francs" : 1.2,
                        "Australian dollars (A$)" : 0.64, "Chinese yuan renminbi (¥)" : 0.14, "Japanese yen (¥)" : 0.0069, "Indian rupees (?)" : 0.012, "Canadian dollars (C$)" : 0.72, 
                        "Bitcoin (btc)" : 103913.38, "Mexican pesos (MXN$)": 0.51,"Russian rubles (?)" : 0.012, "Brazilian reais (R$)" : 0.18}

# Map company sizes to numeric
df_professionals["CompanySize"] = df_professionals["CompanySize"].map(company_size_num)

# Remove entries with missing or unknown company size after mapping
df_professionals = df_professionals.dropna(subset=["CompanySize"])

# Filter out currencies not in dictionary
df_professionals = df_professionals[df_professionals["Currency"].isin(currencies_to_dollar)]

# Convert salary to USD
df_professionals["SalaryUSD"] = df_professionals.apply(
    lambda row: row["Salary"] * currencies_to_dollar[row["Currency"]], axis=1
)


In [23]:
df_professionals.head(10)

,Respondent,Gender,Country,EmploymentStatus,FormalEducation,CompanyType,CompanySize,Currency,Professional,HaveWorkedLanguage,WantWorkLanguage,JobSatisfaction,Salary,SalaryUSD
0,3,Male,United Kingdom,Employed full-time,Bachelor's degree,Publicly-traded corporation,8.0,British pounds sterling (£),Professional developer,Java; PHP; Python,C; Python; Rust,9.0,113750.000000,151287.500000
9,15,Male,United Kingdom,Employed full-time,Professional degree,Publicly-traded corporation,7.0,British pounds sterling (£),Professional developer,Assembly; C; C++,Assembly; C; C++; Python,8.0,100000.000000,133000.000000
14,20,Male,Greece,Employed full-time,Doctoral degree,Venture-funded startup,1.0,Euros (€),Professional developer,Perl; Python,Perl; Python,8.0,NaN,NaN
21,35,Male,Croatia,Employed full-time,Bachelor's degree,"Privately-held limited company, not in startup...",2.0,Euros (€),Professional developer,Assembly; C,C++; Elixir; Python,8.0,14838.709677,16767.741935
23,38,Male,Germany,Employed full-time,Some college/university study without earning ...,Venture-funded startup,4.0,Euros (€),Professional developer,C#; Java; JavaScript,C#; JavaScript,NaN,118279.569892,133655.913978
32,54,Male,United Kingdom,Employed full-time,Bachelor's degree,"Privately-held limited company, not in startup...",1.0,British pounds sterling (£),Professional developer,Java; JavaScript,CoffeeScript; Clojure; Go; JavaScript,7.0,NaN,NaN
33,55,Male,Germany,Employed full-time,Master's degree,I don't know,6.0,Euros (€),Professional developer,Java; JavaScript; PHP,TypeScript,9.0,64516.129032,72903.225806
35,58,Male,Italy,Employed full-time,Master's degree,"Privately-held limited company, not in startup...",4.0,Euros (€),Professional developer,Go; Java,C#; Go; Java; Scala,6.0,43010.752688,48602.150538
38,65,Male,Netherlands,Employed full-time,Master's degree,"Privately-held limited company, not in startup...",8.0,Euros (€),Professional developer,C; C++; C#; Java; JavaScript; Matlab; SQL,Erlang; F#; R; SQL; Swift,5.0,69892.473118,78978.494624
39,66,Male,Poland,Employed full-time,Bachelor's degree,"Privately-held limited company, not in startup...",3.0,Polish zloty (zl),Professional developer,JavaScript,Go; JavaScript,8.0,17777.777778,4800.000000


In [24]:
df_professionals.to_csv("../data/professionals.csv", index=False)